In [ ]:
import os
import sys
import random
import warnings
import cv2

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from tqdm import tqdm
from itertools import chain
from skimage import feature
from skimage.color import rgb2grey
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from skimage.morphology import label

from keras.models import Model, load_model
from keras.layers import Input
from keras.layers.core import Dropout, Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K

import tensorflow as tf

# Set some parameters
IMG_WIDTH = 256
IMG_HEIGHT = 256
IMG_CHANNELS = 3
TRAIN_PATH = '../input/stage1_train/'
TEST_PATH = '../input/stage1_test/'

warnings.filterwarnings('ignore', category=UserWarning, module='skimage')
seed = 42
random.seed = seed
np.random.seed = seed

# Get train and test IDs
train_ids = next(os.walk(TRAIN_PATH))[1]
test_ids = next(os.walk(TEST_PATH))[1]

In [ ]:
def buildUNetModel():
    print("Building UNet Model ...")
    sys.stdout.flush()
    # Build U-Net model
    inputs = Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
    s = Lambda(lambda x: x / 255) (inputs)

    c1 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (s)
    c1 = Dropout(0.1) (c1)
    c1 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c1)
    p1 = MaxPooling2D((2, 2)) (c1)

    c2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p1)
    c2 = Dropout(0.1) (c2)
    c2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c2)
    p2 = MaxPooling2D((2, 2)) (c2)

    c3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p2)
    c3 = Dropout(0.2) (c3)
    c3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c3)
    p3 = MaxPooling2D((2, 2)) (c3)

    c4 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p3)
    c4 = Dropout(0.2) (c4)
    c4 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c4)
    p4 = MaxPooling2D(pool_size=(2, 2)) (c4)

    c5 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p4)
    c5 = Dropout(0.3) (c5)
    c5 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c5)

    u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same') (c5)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u6)
    c6 = Dropout(0.2) (c6)
    c6 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c6)

    u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c6)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u7)
    c7 = Dropout(0.2) (c7)
    c7 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c7)

    u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u8)
    c8 = Dropout(0.1) (c8)
    c8 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c8)

    u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same') (c8)
    u9 = concatenate([u9, c1], axis=3)
    c9 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u9)
    c9 = Dropout(0.1) (c9)
    c9 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c9)

    outputs = Conv2D(1, (1, 1), activation='sigmoid') (c9)

    model = Model(inputs=[inputs], outputs=[outputs])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    print("UNet Model built")
    sys.stdout.flush()    
    return model

def fitAndPredict(model, results, epochs):
    X_train = results['X_train']
    Y_train = results['Y_train']    
    print("Fitting model ...")
    sys.stdout.flush()    


    # Fit model
    earlystopper = EarlyStopping(patience=5, verbose=1)
    checkpointer = ModelCheckpoint('model-dsbowl2018-1.h5', verbose=1, save_best_only=True)
    results = model.fit(X_train, Y_train, validation_split=0.1, batch_size=16, epochs=epochs, 
                        callbacks=[earlystopper, checkpointer])
    #print("Model has been fit")
    #sys.stdout.flush()    

#     print("Predicting on train, val and test ...")
#     sys.stdout.flush()    

#     # Predict on train, val and test
#     model = load_model('model-dsbowl2018-1.h5', custom_objects={'mean_iou': mean_iou})
#     preds_train = model.predict(X_train[:int(X_train.shape[0]*0.9)], verbose=1)
#     preds_val = model.predict(X_train[int(X_train.shape[0]*0.9):], verbose=1)
#     preds_test = model.predict(X_test, verbose=1)

#     # Threshold predictions
#     preds_train_t = (preds_train > 0.5).astype(np.uint8)
#     preds_val_t = (preds_val > 0.5).astype(np.uint8)
#     preds_test_t = (preds_test > 0.5).astype(np.uint8)

#     # Create list of upsampled test masks
#     preds_test_upsampled = []
#     for i in range(len(preds_test)):
#         preds_test_upsampled.append(resize(np.squeeze(preds_test[i]), 
#                                            (sizes_test[i][0], sizes_test[i][1]), 
#                                            mode='constant', preserve_range=True))
        
#     print("Perform a sanity check on some random training samples")
#     sys.stdout.flush()         
#     ix = random.randint(0, len(preds_train_t))
#     imshow(X_train[ix])
#     plt.show()
#     imshow(np.squeeze(Y_train[ix]))
#     plt.show()
#     #print(preds_train_t)
#     #print(ix)
#     imshow(np.squeeze(preds_train_t[ix]))
#     plt.show()

#     print("Perform a sanity check on some random validation samples")
#     sys.stdout.flush()         
#     ix = random.randint(0, len(preds_val_t))
#     imshow(X_train[int(X_train.shape[0]*0.9):][ix])
#     plt.show()
#     imshow(np.squeeze(Y_train[int(Y_train.shape[0]*0.9):][ix]))
#     plt.show()
#     imshow(np.squeeze(preds_val_t[ix]))
#     plt.show()

def checkTrainingData(ids, trains, masks):
    # Check training data
    ix = random.randint(0, len(ids) - 1)
    print("Image")
    imshow(trains[ix])
    plt.show()
    print("Mask")    
    imshow(np.squeeze(masks[ix]))
    plt.show()

In [ ]:
def testModel(images, modelBuildingFunction, epochs):
    print("Images pre-processed, moving on to fitting and predictions ...")
    sys.stdout.flush()
    fitAndPredict(
        modelBuildingFunction(),
        images,
        epochs
    )

In [ ]:
def preprocessImagesOne(numOfSamples=5):
    # Get and resize train images and masks
    X_train = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
    Y_train = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)
    print('Getting and resizing train images and masks ... ')
    sys.stdout.flush()
    for n, id_ in tqdm(enumerate(train_ids), total=len(train_ids)):
        path = TRAIN_PATH + id_
        img = imread(path + '/images/' + id_ + '.png')[:,:,:IMG_CHANNELS]

        img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)

        X_train[n] = img
        mask = np.zeros((IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)
        for mask_file in next(os.walk(path + '/masks/'))[2]:
            mask_ = imread(path + '/masks/' + mask_file)
            mask_ = np.expand_dims(resize(mask_, (IMG_HEIGHT, IMG_WIDTH), mode='constant', 
                                          preserve_range=True), axis=-1)
            mask = np.maximum(mask, mask_)
        Y_train[n] = mask

    print('Done!')
    # Check training data
    for i in range(numOfSamples):
        checkTrainingData(train_ids, X_train, Y_train)

    return { 'X_train': X_train, 'Y_train': Y_train }

In [ ]:
def preprocessImagesTwo(numOfSamples=5):
    # Get and resize train images and masks
    X_train = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
    Y_train = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)
    print('Getting and resizing train images and masks ... ')
    sys.stdout.flush()
    for n, id_ in tqdm(enumerate(train_ids), total=len(train_ids)):
        path = TRAIN_PATH + id_
        img = imread(path + '/images/' + id_ + '.png')[:,:,:IMG_CHANNELS]

        #print(img)
        img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
        #img_edges = cv2.Canny(img, 100, 200)
        img_edges = 255*feature.canny(rgb2grey(img))

    #    img_edges = img_edges[..., np.newaxis]
        
        img_edges = np.stack((img_edges,)*3, -1)
     #   print(img_edges)
#         img_edges = resize(img_edges, (IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))        
#         print(stacked_img)
        #imshow(img_edges)
        #plt.show()
        
        
        
        X_train[n] = img_edges
        mask = np.zeros((IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)
        for mask_file in next(os.walk(path + '/masks/'))[2]:
            mask_ = imread(path + '/masks/' + mask_file)
            mask_ = np.expand_dims(resize(mask_, (IMG_HEIGHT, IMG_WIDTH), mode='constant', 
                                          preserve_range=True), axis=-1)
            mask = np.maximum(mask, mask_)
        Y_train[n] = mask

    print('Done!')
    # Check training data
    for i in range(numOfSamples):
        checkTrainingData(train_ids, X_train, Y_train)

    return { 'X_train': X_train, 'Y_train': Y_train }

In [ ]:
def preprocessImagesThree(numOfSamples=5):
    # Get and resize train images and masks
    X_train = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
    Y_train = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)
    print('Getting and resizing train images and masks ... ')
    sys.stdout.flush()
    for n, id_ in tqdm(enumerate(train_ids), total=len(train_ids)):
        path = TRAIN_PATH + id_
        img = imread(path + '/images/' + id_ + '.png')[:,:,:IMG_CHANNELS]

        #print(img)
        img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
        img_edges = 255*feature.canny(rgb2grey(img))
        img_edges = np.stack((img_edges,)*3, -1)
#         old_set = [[0, 1], [4, 5]]
#         new_set = [[2, 7], [0, 1]]
        blended_img = (np.array(img_edges) + np.array(img)) / 2

        imshow(blended_img)
        plt.show()
        #imshow(img)
        #plt.show()
        #imshow(img_edges)
        #plt.show()
        
        
        
        X_train[n] = img_edges
        mask = np.zeros((IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)
        for mask_file in next(os.walk(path + '/masks/'))[2]:
            mask_ = imread(path + '/masks/' + mask_file)
            mask_ = np.expand_dims(resize(mask_, (IMG_HEIGHT, IMG_WIDTH), mode='constant', 
                                          preserve_range=True), axis=-1)
            mask = np.maximum(mask, mask_)
        Y_train[n] = mask

    print('Done!')
    # Check training data
    for i in range(numOfSamples):
        checkTrainingData(train_ids, X_train, Y_train)

    return { 'X_train': X_train, 'Y_train': Y_train }

#preprocessImagesThree(0)
#print('done')

In [ ]:
images = preprocessImagesOne(3)
testModel(images, buildUNetModel, epochs=25)

In [ ]:
images = preprocessImagesTwo(3)
testModel(images, buildUNetModel, epochs=5)

In [ ]:
images = preprocessImagesThree(3)
testModel(images, buildUNetModel, epochs=5)